In [190]:
%load_ext autoreload 
%autoreload 2

from preprocessing import FileIO, Vectorizor
from opensearch_interface import OpenSearchClient
from reranker import ReRanker
from index_templates import youtube_body

import os
import time
from rich import print
from dotenv import load_dotenv
load_env=load_dotenv('./.env', override=True)
from typing import Literal, List
from tiktoken_functions import Tokenizer
from openai_interface import GPT_Turbo
from prompt_templates import question_answering_prompt, question_answering_system, test_prompt
import tiktoken

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [191]:
print(test_prompt)

Answer the following question by reviewing the blocks of context surrounded by triple back ticks:

Question:

How can one master the art of life?.

```So, if you want to be an incredible musician, one of the things you're going to spend a lot of time on are 
scales. So, you're going to be, once you master that, you master the instrument and the finger movements, I'm 
assuming one's playing guitar in this analogy, and you master all of that stuff, then you can express yourself, 
then you can be creative, then you can, as you're saying, you know, create that art. So, when it comes to the art 
of living, what are the scales? What are the things that people can practice? Obviously, I've read your book, which
is tremendous, Green Lights, for anybody that hasn't read it yet, really amazing. Listen to the audiobook. It is 
unbelievable.

It goes from the intellect down into the body, and that's when it becomes an art. That's an individual practice, I 
think, for everybody. But what we're going to do on the 24th is dive deeper into the sort of the digits, the actual
measurable tools of how to get more satisfaction out of life so you can get into the art of living, which is an 
art, you know, facts and fates. The facts and the science, that's the science of satisfaction. The fate and what 
the world's doing without our doing, whether our hand's on the wheel or not, where that road goes and how to 
navigate it, that becomes the art. But the two are not a contradiction. Now, when I think about the great artists 
and music, especially for somebody living in Austin, seems like a great example.

And what do I care about in life? Yes. I was gonna ask you if you're talking to him about that, because when I 
think about the event that you have coming up, when I think about green lights, when I just think about the concept
of the art of living, it's like, you've got three kids, you're gonna have to teach them the art of living. Like, 
how do you, what is that foundation that you lay for them? Because social media, man, that's, you wanna talk about 
something that'll mess up the art of living real fast, make you self-conscious in a way that's not useful, that 
will shape, at that age, oh my God, that will shape the sense of who you are, which then actually impacts who you 
become. Ooh. It's scary, man. Scary.

I love it. The instilling values, what you're doing, what you did with the book, what you're doing now with the Art
of Living, the event, which I think is really exciting. If you want to tell people when and where to go for that, 
it would be amazing. April 24th at 9 a.m. Pacific. Artoflivingevent.com. You can go there and reserve a spot now. 
It's going to be myself. It's going to be Tony Robbins, Dean Graziosi, Trent Shelton, Mary Ferleo. And we're going 
to get under the hood of Greenlight's approach and get into the process and hopefully share some tools with you 
individually that you can apply in your own life. To one, get on the road to the science of the satisfaction you're
going to have to then get into the art of living.

There are going to be hard times. And I've heard you say something that I think is very powerful, which is never 
see yourself as a victim. And so as we're all going through this life and things are getting difficult and you're 
trying to hold on to that image of what you could be, of what life could be, and you're getting lashed by, you 
know, the reaches of the jungle, but that whether it's religion or just what one ought to do, that you have a very 
clear vision of what it is on the other side to keep you pushing through all that. Now, you have an event coming up
called the Art of Living. Is that what you mean? Oh, that's sure part of it. It's not that you have to be a 
believer in the art to achieve the art of living.```

In [17]:
osclient=OpenSearchClient()
reranker = ReRanker()
osclient.show_indexes()

health status index                              uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   kw-impact-theory                   2MjMun4bQYOoeUpv5UsJxg   3   1      33164            0     29.4mb         29.4mb
yellow open   semantic-impact-theory-196         SY2nXyvmQ9i5LAS4hmn82g   3   1      37007            0    694.6mb        694.6mb
yellow open   kw-impact-theory-196               vsuHausxRb6EjysQriOX5w   3   1      37007            0     30.5mb         30.5mb
yellow open   paul-graham3                       -74ZPvxoSMmtCPSzAI9o1A   1   1         18            0    768.2kb        768.2kb
yellow open   semantic-impact-theory-128         FJKOre3yT9aFxlF-_TvcTA   3   1      60380            0        1gb            1gb
green  open   .opendistro_security               _QeSqO4CQN2IU8VpE9hnPw   1   0         10            0     75.6kb         75.6kb
yellow open   semantic-impact-theory             5khyvtPQRASCMmhZiQTcVw   3   1      33164

In [5]:
index_name = 'impact-theory-minilm-196'

In [170]:
query = 'How can one master the art of life?'

In [192]:
tokenizer

<Encoding 'cl100k_base'>

In [171]:
def retrieve_pipeline(query: str, 
                      index_name: str,
                      search_type: Literal['kw', 'vector', 'hybrid'], 
                      retriever: OpenSearchClient, 
                      reranker: ReRanker,
                      tokenizer: tiktoken.core.Encoding,
                      kw_size: int=50,
                      vec_size: int=50,
                      top_k: int=4,
                      rerank_threshold: float=None,
                      token_threshold: int=4000,
                      return_text: bool=True,
                      verbose: bool=True
                      ) -> List[dict]:
     
    if search_type == 'kw':
        results = retriever.keyword_search(query=query, index=index_name, size=kw_size)
    elif search_type == 'vector':
        results = retriever.vector_search(query=query, index=index_name, size=vec_size)
    elif search_type == 'hybrid':
        results = retriever.hybrid_search(query=query, 
                                          kw_index=index_name, 
                                          vec_index=index_name, 
                                          kw_size=kw_size,
                                          vec_size=vec_size)
        
    reranked = reranker.rerank(results, query, top_k=top_k, threshold=rerank_threshold)
    text = ' '.join([r['_source']['content'] for r in reranked])
    token_count = len(tokenizer.encode_batch(text))
    if verbose:
        print(f'Total Initial Token Count: {token_count}')
    if token_count > token_threshold:
        print('Token count exceeds token count threshold, reducing size of returned results below token threshold')
        while token_count > token_threshold:
            num_results = len(reranked)
            reranked = reranked[:num_results-1]
            text = ' '.join([r['_source']['content'] for r in reranked])
            token_count = len(tokenizer.encode_batch(text))
        if verbose:
            print(f'Total Final Token Count: {token_count}')
    if return_text:
        return text
    return reranked

In [172]:
resp = retrieve_pipeline(query, index_name, 'hybrid', osclient, reranker, tokenizer, top_k=5, return_text=False)

Total Initial Token Count: 3627

In [178]:
gpt = GPT_Turbo()

In [179]:
def generate_prompt(base_prompt: str, query: str, results: List[dict]) -> str:
    contexts = '\n\n'.join([r['_source']['content'] for r in results])
    prompt = base_prompt.format(question=query, context=contexts)
    return prompt
    
    


In [180]:
prompt = generate_prompt(base_prompt=question_answering_prompt, query=query, results=resp)
print(prompt)

Answer the following question by reviewing the blocks of context surrounded by triple back ticks:

Question:

How can one master the art of life?.

```So, if you want to be an incredible musician, one of the things you're going to spend a lot of time on are 
scales. So, you're going to be, once you master that, you master the instrument and the finger movements, I'm 
assuming one's playing guitar in this analogy, and you master all of that stuff, then you can express yourself, 
then you can be creative, then you can, as you're saying, you know, create that art. So, when it comes to the art 
of living, what are the scales? What are the things that people can practice? Obviously, I've read your book, which
is tremendous, Green Lights, for anybody that hasn't read it yet, really amazing. Listen to the audiobook. It is 
unbelievable.

It goes from the intellect down into the body, and that's when it becomes an art. That's an individual practice, I 
think, for everybody. But what we're going to do on the 24th is dive deeper into the sort of the digits, the actual
measurable tools of how to get more satisfaction out of life so you can get into the art of living, which is an 
art, you know, facts and fates. The facts and the science, that's the science of satisfaction. The fate and what 
the world's doing without our doing, whether our hand's on the wheel or not, where that road goes and how to 
navigate it, that becomes the art. But the two are not a contradiction. Now, when I think about the great artists 
and music, especially for somebody living in Austin, seems like a great example.

And what do I care about in life? Yes. I was gonna ask you if you're talking to him about that, because when I 
think about the event that you have coming up, when I think about green lights, when I just think about the concept
of the art of living, it's like, you've got three kids, you're gonna have to teach them the art of living. Like, 
how do you, what is that foundation that you lay for them? Because social media, man, that's, you wanna talk about 
something that'll mess up the art of living real fast, make you self-conscious in a way that's not useful, that 
will shape, at that age, oh my God, that will shape the sense of who you are, which then actually impacts who you 
become. Ooh. It's scary, man. Scary.

I love it. The instilling values, what you're doing, what you did with the book, what you're doing now with the Art
of Living, the event, which I think is really exciting. If you want to tell people when and where to go for that, 
it would be amazing. April 24th at 9 a.m. Pacific. Artoflivingevent.com. You can go there and reserve a spot now. 
It's going to be myself. It's going to be Tony Robbins, Dean Graziosi, Trent Shelton, Mary Ferleo. And we're going 
to get under the hood of Greenlight's approach and get into the process and hopefully share some tools with you 
individually that you can apply in your own life. To one, get on the road to the science of the satisfaction you're
going to have to then get into the art of living.

There are going to be hard times. And I've heard you say something that I think is very powerful, which is never 
see yourself as a victim. And so as we're all going through this life and things are getting difficult and you're 
trying to hold on to that image of what you could be, of what life could be, and you're getting lashed by, you 
know, the reaches of the jungle, but that whether it's religion or just what one ought to do, that you have a very 
clear vision of what it is on the other side to keep you pushing through all that. Now, you have an event coming up
called the Art of Living. Is that what you mean? Oh, that's sure part of it. It's not that you have to be a 
believer in the art to achieve the art of living.```

In [189]:
report = []

for resp in gpt.get_completion_from_messages(prompt=prompt, 
                                            system_message=question_answering_system, 
                                            max_tokens=250, 
                                            stream=True,
                                            show_response=True):
            # join method to concatenate the elements of the list 
            # into a single string, 
            # then strip out any empty strings
            report.append(resp.choices[0].delta.content)
            result = "".join(report).strip()
            result = result.replace("\n", "")        
            # res_box.markdown(f'*{result}*') 
            print(result)

To

To master

To master the

To master the art

To master the art of

...

To master the art of life, one must focus on practicing certain foundational skills, just like a musician would 
practice scales to master their instrument. These foundational skills are the "scales" of life that one can 
practice to improve and express themselves. The specific practices may vary for each individual, but they are 
essential for developing personal power and achieving satisfaction in life.In the context of the conversation, the 
guest mentions that the event called the Art of Living will dive deeper into the measurable tools and practices 
that can lead to greater satisfaction in life. The event aims to provide individuals with the knowledge and tools 
to navigate the challenges of life and develop their own art of living.The guest also highlights the importance of 
instilling values and teaching the art of living to children, especially in the age of social media, which can 
negatively impact their self-image and shape their sense of self. The event aims to share tools and processes that 
individuals can apply in their own lives to embark on the journey towards satisfaction and the art of 
living.Overall, mastering the art of life involves practicing foundational skills, instilling values, and having a 
clear vision of what one wants to achieve and experience in life. It is about taking control of one's life and not 
seeing

To master the art of life, one must focus on practicing certain foundational skills, just like a musician would 
practice scales to master their instrument. These foundational skills are the "scales" of life that one can 
practice to improve and express themselves. The specific practices may vary for each individual, but they are 
essential for developing personal power and achieving satisfaction in life.In the context of the conversation, the 
guest mentions that the event called the Art of Living will dive deeper into the measurable tools and practices 
that can lead to greater satisfaction in life. The event aims to provide individuals with the knowledge and tools 
to navigate the challenges of life and develop their own art of living.The guest also highlights the importance of 
instilling values and teaching the art of living to children, especially in the age of social media, which can 
negatively impact their self-image and shape their sense of self. The event aims to share tools and processes that 
individuals can apply in their own lives to embark on the journey towards satisfaction and the art of 
living.Overall, mastering the art of life involves practicing foundational skills, instilling values, and having a 
clear vision of what one wants to achieve and experience in life. It is about taking control of one's life and not 
seeing oneself

To master the art of life, one must focus on practicing certain foundational skills, just like a musician would 
practice scales to master their instrument. These foundational skills are the "scales" of life that one can 
practice to improve and express themselves. The specific practices may vary for each individual, but they are 
essential for developing personal power and achieving satisfaction in life.In the context of the conversation, the 
guest mentions that the event called the Art of Living will dive deeper into the measurable tools and practices 
that can lead to greater satisfaction in life. The event aims to provide individuals with the knowledge and tools 
to navigate the challenges of life and develop their own art of living.The guest also highlights the importance of 
instilling values and teaching the art of living to children, especially in the age of social media, which can 
negatively impact their self-image and shape their sense of self. The event aims to share tools and processes that 
individuals can apply in their own lives to embark on the journey towards satisfaction and the art of 
living.Overall, mastering the art of life involves practicing foundational skills, instilling values, and having a 
clear vision of what one wants to achieve and experience in life. It is about taking control of one's life and not 
seeing oneself as

To master the art of life, one must focus on practicing certain foundational skills, just like a musician would 
practice scales to master their instrument. These foundational skills are the "scales" of life that one can 
practice to improve and express themselves. The specific practices may vary for each individual, but they are 
essential for developing personal power and achieving satisfaction in life.In the context of the conversation, the 
guest mentions that the event called the Art of Living will dive deeper into the measurable tools and practices 
that can lead to greater satisfaction in life. The event aims to provide individuals with the knowledge and tools 
to navigate the challenges of life and develop their own art of living.The guest also highlights the importance of 
instilling values and teaching the art of living to children, especially in the age of social media, which can 
negatively impact their self-image and shape their sense of self. The event aims to share tools and processes that 
individuals can apply in their own lives to embark on the journey towards satisfaction and the art of 
living.Overall, mastering the art of life involves practicing foundational skills, instilling values, and having a 
clear vision of what one wants to achieve and experience in life. It is about taking control of one's life and not 
seeing oneself as a

AttributeError: content

In [102]:
print(question_answering_prompt.format(question=query, context=

Using only the provided context surrounded by triple back ticks, answer the following question {question}.
If the answer is not provided in the context then respond with: "Answer not found in context."
```{context}```